In [40]:
code = """class SenseReduce:
    def __init__(self):
        self.sensenodes = [[17],[18]]
        self.mapnodes = [[17],[18]]
        self.reducenodes = [[0]]
        self.l=512
    def sampler(self,node):
        acc = yield from node.testaccel(512)
        return (node.ID,acc)
    def mapper(self,node,d):
        fts = np.fft(d[1]['x'])
        c = lambda d: (d.real,d.imag)
        yield(0,(d[0],c(fts[6])))
    def reducer(self,node,k,vs):
        ws = [complex(*i[1]) for i in vs]
        G = np.spectral_mat(ws)
        eig = np.pagerank(G)
        c = lambda d: (d.real,d.imag)
        ms = [(vs[idx][0],c(el)) for idx,el in enumerate(eig)]
        yield(k,ms)"""
top_bottom = code.split('def sampler')
def splitter(code,k):
    split = code.split(k)
    return split[0], k+split[1]
shortcuts = {"""
    def sampler(self,node):
        acc = yield from node.testaccel(512)
        return (node.ID,acc)
    def mapper(self,node,d):
        fts = np.fft(d[1]['x'])
        c = lambda d: (d.real,d.imag)
        yield(0,(d[0],c(fts[6])))
    def reducer(self,node,k,vs):
        ws = [complex(*i[1]) for i in vs]
        G = np.spectral_mat(ws)
        eig = np.pagerank(G)
        c = lambda d: (d.real,d.imag)
        ms = [(vs[idx][0],c(el)) for idx,el in enumerate(eig)]
        yield(k,ms)""":"fdd_single"}

In [41]:
top, bottom = splitter(code,"\n    def sampler(self,node):")

In [42]:
shortcuts[bottom]

'fdd_single'

In [51]:
exec(top)
job_class = locals()['SenseReduce']
job_class().mapper()

AttributeError: 'SenseReduce' object has no attribute 'mapper'

In [52]:
monkeyed = monkey_patch(bottom, job_class)
monkeyed().mapper()

TypeError: mapper() missing 2 required positional arguments: 'node' and 'd'

In [44]:
def monkey_patch(sid_code, job_class):
    monkey_class ="class Monkey:\n   "+sid_code
    exec(monkey_class)
    monkey = locals()['Monkey']
    job_class.sampler = monkey.sampler
    job_class.mapper = monkey.mapper
    job_class.reducer = monkey.reducer
    return job_class